# Import libraries

In [28]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2 as cv2
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

# Preprocessing

In [29]:
# Define the path to the dataset directory
dataset_dir = 'CrossMatch_Sample_DB'

# Lists to store image data and labels
image_data = []
labels = []

# Iterate through the dataset and load images and labels
for filename in os.listdir(dataset_dir):
    if filename.lower().endswith('.tif'):  # Check for TIF files, case-insensitive
        # Extract labels from the filename
        person_id, finger_id, scan_num = filename.split('_')
        person_id = int(person_id)  # Convert person ID to an integer
        finger_id = int(finger_id)  # Convert finger ID to an integer

        # Load the image
        img_path = os.path.join(dataset_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            image_data.append(img)
            labels.append((person_id, finger_id))

# Now 'image_data' contains the loaded images, and 'labels' contains the corresponding labels


In [30]:
print(len(image_data))

408


In [31]:
# Convert the 'labels' list to a NumPy array
labels_array = np.array(labels)

# Extract individual components: person IDs, finger IDs, and scan numbers
person_ids = np.unique(labels_array[:, 0])
finger_ids = np.unique(labels_array[:, 1])

num_unique_persons = len(person_ids)
num_unique_fingers = len(finger_ids)

print("Number of unique persons:", num_unique_persons)
print(person_ids)
print("Number of unique fingers:", num_unique_fingers)
print(finger_ids)

Number of unique persons: 9
[12 13 17 22 27 45 47 57 76]
Number of unique fingers: 6
[3 4 5 6 7 8]


In [32]:
# Initialize HOG descriptor
hog = cv2.HOGDescriptor()

# Extract features using HOG for each image
hog_features = []
for img in image_data:
    hog_img = cv2.resize(img, (64, 128))  # Resize image to a common size for HOG
    features = hog.compute(hog_img)
    hog_features.append(features.flatten())

# Now 'hog_features' contains the extracted HOG features


In [34]:
from sklearn.preprocessing import LabelEncoder

# Combine person_id and finger_id into a single label
combined_labels = ['{}_{}'.format(person_id, finger_id) for person_id, finger_id in labels]

# Use LabelEncoder to transform the combined labels into encoded integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(combined_labels)

# Now 'encoded_labels' contains the encoded target labels


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


features = hog_features  

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# Initialize SVM classifier
svm = SVC(kernel='linear')

# Train the SVM model
svm.fit(X_train, y_train)

# Predict on the test set
predictions = svm.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.7926829268292683
